In [1]:
import sys
sys.path.append('..')

import gym

import torch
import torch.nn as nn

from models.ddpg import DDPG

In [2]:
class Actor(nn.Module):
    def __init__(self, nstates: int, nactions: int):
        super(Actor, self).__init__()
                
        self.fc1 = nn.Linear(nstates, 400)
        self.fc2 = nn.Linear(400, 300)
        self.fc3 = nn.Linear(300, nactions)
        
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()
    
    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        x = torch.tanh(self.fc3(x))
        
        return x

In [3]:
class Critic(nn.Module):
    def __init__(self, nstates: int, nactions: int):
        super(Critic, self).__init__()
                
        self.fc1 = nn.Linear(nstates, 400)
        self.fc2 = nn.Linear(nactions + 400, 300)
        self.fc3 = nn.Linear(300, 1)
        
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()
                
    def forward(self, xs):
        x, a = xs
        
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(torch.cat([x, a], 1)))
        x = self.fc3(x)
        
        return x

In [4]:
cri_lr = 1e-3
act_lr = 5e-4

tau = 5e-3

nstates = 17
nactions = 6

n_epis = 500
n_epochs = 200
n_rollout = 10

act_range = (-1, 1)

In [5]:
env = gym.make("HalfCheetah-v4", render_mode='human')

In [6]:
act = Actor(nstates, nactions)
act_opt = torch.optim.Adam(act.parameters(), lr=act_lr)

cri = Critic(nstates, nactions)
cri_opt = torch.optim.Adam(cri.parameters(), lr=cri_lr)

In [7]:
ddpg = DDPG(env, nactions, act, act_opt, cri, cri_opt, act_noise=0.2, act_range=act_range)

In [ ]:
ddpg.train(n_epis, 1, n_rollout)

c:\Users\esder\AppData\Local\Programs\Python\Python38\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


: 

: 

In [8]:
env.close()